<a href="https://colab.research.google.com/github/kirill-kozlitin/VTB/blob/main/check_autologreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Тетрадка представляет собой пример построения скоринговой модели с помощью инструментов автологрега.

## 1. Загрузка данных

In [1]:
# Подключение к облачному хранилищу
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Установка необходимых для работы библиотеки пакетов
!pip install -q xlsxwriter
!pip install -q tsfresh
!pip install -q optbinning

In [3]:
# Импорт библиотек
import pandas as pd
import sys

path = 'drive/MyDrive/VTB/'
sys.path.append(path)

import vtb_mlkit
from vtb_mlkit.scorekit.model import LogisticRegressionModel
from vtb_mlkit.scorekit.data import DataSamples


(CVXPY) Jul 29 01:50:18 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jul 29 01:50:18 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [4]:
# загрузка датасета
data = pd.read_csv(path + 'loan_data.csv', sep=';',)# nrows=10**5)
print('Размер датасета:', data.shape)

Размер датасета: (999988, 51)


In [5]:
# загрузка описания фичей
description = pd.read_excel(path + 'lending_club_description.xlsx')
description.index = description.index.astype(str) # костыль. Должно работать на любых данных

## Обучение

In [6]:
# Создаем объект класса DataSample
ds = DataSamples(samples={'train': data.copy()}, target='target', feature_descriptions=description, time_column='date')

[INFO] [2023-07-29 13:50:27] ---------------------------------------------------------------- Creating DataSamples ----------------------------------------------------------------
[INFO] [2023-07-29 13:50:28] Selected 49 features: ['int_rate', 'out_prncp', 'out_prncp_inv', 'recoveries', 'collection_recovery_fee', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'num_rev_tl_bal_gt_0', 'num_actv_rev_tl', 'fico_range_high', 'fico_range_low', 'acc_open_past_24mths', 'total_bal_ex_mort', 'num_actv_bc_tl', 'revol_bal', 'num_sats', 'open_acc', 'num_tl_op_past_12m', 'num_op_rev_tl', 'total_il_high_credit_limit', 'inq_last_6mths', 'mo_sin_rcnt_tl', 'num_il_tl', 'mort_acc', 'num_bc_sats', 'mo_sin_rcnt_rev_tl_op', 'avg_cur_bal', 'total_acc', 'mo_sin_old_rev_tl_op', 'total_bc_limit', 'annual_inc', 'tot_hi_cred_lim', 'delinq_2yrs', 'num_accts_ever_120_pd', 'num_bc_tl', 'tot_coll_amt', 'pub_rec', 'pub_rec_bankruptcies', 'pct_tl_nvr_dlq', 'num_tl_90g_dpd_24m', 'collections_12_mths_ex_med

In [7]:
# На основании существующих фичей генерируем дополнительные новые
ds.feature_generator(features=None, prebinning=True, chunk_size=10000, gini_threshold=10, operands=True,
                     lags=True, aggs=None, corr_method='spearman', corr_thres=0.80)

[INFO] [2023-07-29 13:51:19] Creating combinations...


100%|██████████| 18/18 [00:21<00:00,  1.21s/it]

[INFO] [2023-07-29 13:51:41] 18 features were generated, 0 of them passed the tests and were added to sample train


In [8]:
# Разбиваем выборку на трейн и тест
ds.samples_split(df=None, test_size=.3, validate_size=0, split_type='oos', stratify=True, id_column=None)

[INFO] [2023-07-29 13:51:43] Actual parts of samples after samples split:
      train  Test
part    0.7   0.3


In [9]:
# Создаем объект класса LogisticRegressionModel
logreg = LogisticRegressionModel(clf=None,            # классификатор модели (должен иметь метод fit() и атрибуты coef_, intercept_). При None выбирается SGDClassifier(alpha=0.001, loss='log', max_iter=100)
                                 ds=ds,               # Привязанный к модели ДатаСэмпл. Если задан, то он по умолчанию будет использоваться во всех методах
                                 transformer=None,    # объект класса WOE для предварительной трансформации факторов
                                 round_digits=3,      # округление коэффициентов до этого кол-ва знаков после запятой
                                 name='loan',      # название модели
                                )


[INFO] [2023-07-29 13:51:43] Chosen model classifier is SGDClassifier(alpha=0.001, loss='log', random_state=0)


In [ ]:
# Запускаем процесс обучения модели
logreg.auto_logreg(data=None, target=None, time_column=None, id_column=None, feature_descriptions=description, n_jobs=None,
                   result_folder='', method='opt', validate=False, out='auto_model.xlsx', save_model='auto_model.json')

[INFO] [2023-07-29 13:51:44] ------------------------------------------------------------------------ SFA -------------------------------------------------------------------------
[INFO] [2023-07-29 13:51:44] Performing autobinning with parameters space of size 1...
[INFO] [2023-07-29 13:51:44] Using parameters set 1/1: {'method': 'opt', 'max_n_bins': 10, 'min_bin_size': 0.05, 'criterion': 'entropy', 'scoring': 'neg_log_loss', 'max_depth': 5, 'solver': 'cp', 'divergence': 'iv', 'WOEM_on': True, 'WOEM_woe_threshold': 0.05, 'WOEM_with_missing': False, 'SM_on': False, 'SM_target_threshold': 5, 'SM_size_threshold': 100, 'G_on': False, 'G_gini_threshold': 5, 'G_gini_decrease_threshold': 0.25, 'G_gini_increase_restrict': False, 'G_with_test': False, 'BL_on': True, 'BL_allow_Vlogic_to_increase_gini': 10, 'WOEO_on': True, 'WOEO_all_samples': False, 'verbose': False}
[INFO] [2023-07-29 13:51:44] Processing 49 features on 1 CPU...


 20%|██        | 10/49 [00:43<03:44,  5.77s/it]

In [ ]:
ds.feature_descriptions[['Название', 'Описание']].head(10)

In [ ]:
# классификатор модели (должен иметь метод fit() и атрибуты coef_, intercept_)
logreg.clf

In [ ]:
# словарь коэффициентов
logreg.coefs

In [ ]:
# константа логрега
logreg.intercept

In [ ]:
logreg.report(ds=None, out='report.xlsx', sheet_name=None, pvalue_threshold=0.05, verbose=False, add_figs=None, gini_df=None,
              plotbins_config=None, metrics=None, metrics_cv=None, cv=None)